In [1]:
# nuclio: ignore
import nuclio

In [2]:
# Copyright 2018 Iguazio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx


def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [3]:
# nuclio: end-code

### save

In [4]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset", kind="job", with_doc=True,
                      handler=load_dataset, image="mlrun/ml-models:test")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["datagen", "filesutils"]
fn.metadata.labels = {"author": "yjb"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-20 18:55:40,104 saving function: load-dataset, tag: latest
[mlrun] 2020-04-20 18:55:40,159 function spec saved to path: function.yaml


## tests

In [5]:
from mlrun import import_function, mount_v3io, NewTask

func = import_function("hub://load_dataset").apply(mount_v3io())
# func = import_function("function.yaml").apply(mlrun.mount_v3io())

task_params = {"name"          : "tasks load toy dataset", 
               "params"        : {"dataset"     : "wine"}}

### run remotely

In [6]:
run = func.run(NewTask(**task_params), artifact_path="/User/artifacts")

[mlrun] 2020-04-20 18:55:42,791 starting run tasks load toy dataset uid=da13512eaf6f4ae6a4b4482b2e43a6d4  -> http://mlrun-api:8080
[mlrun] 2020-04-20 18:55:43,000 Job is running in the background, pod: tasks-load-toy-dataset-4lj5v
[mlrun] 2020-04-20 18:55:47,908 log artifact wine at /User/artifacts/wine.parquet, size: 18006, db: Y

[mlrun] 2020-04-20 18:55:47,946 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...2e43a6d4,0,Apr 20 18:55:47,completed,tasks load toy dataset,host=tasks-load-toy-dataset-4lj5vkind=jobowner=iguaziov3io_user=iguazio,,dataset=wine,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run da13512eaf6f4ae6a4b4482b2e43a6d4  , !mlrun logs da13512eaf6f4ae6a4b4482b2e43a6d4 
[mlrun] 2020-04-20 18:55:49,261 run executed, status=completed


### or locally

In [7]:
from mlrun import run_local

In [8]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset},
              artifact_path="/User/artifacts")

[mlrun] 2020-04-20 18:55:49,294 starting run mlrun-02600e-load_dataset uid=299e4dda6208471dae90cb1217aa6cfe  -> http://mlrun-api:8080
[mlrun] 2020-04-20 18:55:49,946 log artifact wine at /User/artifacts/wine.parquet, size: 18006, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...17aa6cfe,0,Apr 20 18:55:49,completed,mlrun-02600e-load_dataset,v3io_user=iguaziokind=handlerowner=iguaziohost=jupyter-iguazio-fbcf6f67b-bcshp,dataset,,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run 299e4dda6208471dae90cb1217aa6cfe --project default , !mlrun logs 299e4dda6208471dae90cb1217aa6cfe --project default
[mlrun] 2020-04-20 18:55:50,026 run executed, status=completed
[mlrun] 2020-04-20 18:55:50,034 starting run mlrun-9a9351-load_dataset uid=e5f18fc84ca34436b95e39053ae0a136  -> http://mlrun-api:8080
[mlrun] 2020-04-20 18:55:50,117 log artifact iris at /User/artifacts/iris.parquet, size: 5049, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...3ae0a136,0,Apr 20 18:55:50,completed,mlrun-9a9351-load_dataset,v3io_user=iguaziokind=handlerowner=iguaziohost=jupyter-iguazio-fbcf6f67b-bcshp,dataset,,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run e5f18fc84ca34436b95e39053ae0a136 --project default , !mlrun logs e5f18fc84ca34436b95e39053ae0a136 --project default
[mlrun] 2020-04-20 18:55:50,167 run executed, status=completed
[mlrun] 2020-04-20 18:55:50,175 starting run mlrun-4b3025-load_dataset uid=ba803759a30246e4a783e4f141ac106d  -> http://mlrun-api:8080
[mlrun] 2020-04-20 18:55:50,416 log artifact breast_cancer at /User/artifacts/breast_cancer.parquet, size: 146754, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...41ac106d,0,Apr 20 18:55:50,completed,mlrun-4b3025-load_dataset,v3io_user=iguaziokind=handlerowner=iguaziohost=jupyter-iguazio-fbcf6f67b-bcshp,dataset,,,breast_cancer


to track results use .show() or .logs() or in CLI: 
!mlrun get run ba803759a30246e4a783e4f141ac106d --project default , !mlrun logs ba803759a30246e4a783e4f141ac106d --project default
[mlrun] 2020-04-20 18:55:50,534 run executed, status=completed
